In [6]:
import json
import PySAM.Pvwattsv8 as pv
import pandas as pd

# USER INPUTS
# JSON file should be saved to the same folder as the ipynb file.
json_file_name = 'SAM_PV_Model_Template_pvwattsv8.json'
generator_inputs_location = 'C:\\Users\\fwagg\\OneDrive\\Desktop\\Renewable Energy\\Project 2 - 100% Renewable SD\\SAM\\PV_Generator_Input_Data.csv'
PV_weather_file_location = 'C:\\Users\\fwagg\\OneDrive\\Desktop\\Renewable Energy\\Project 2 - 100% Renewable SD\\SAM\\PV_WeatherFile.csv'
output_data_location = 'C:\\Users\\fwagg\\OneDrive\\Desktop\\Renewable Energy\\Project 2 - 100% Renewable SD\\SAM\\Total_PV_Output_Timeseries.csv'

# Create a new instance of a pySAM model
pv_model = pv.new()

# Declare all variables as strings. Spaces must be replaced with '+', i.e., change 'John Smith' to 'John+Smith'.
# Define the lat, long of the location and the year
api_key = 'zRso1hhdHGyzE1JXUAVz5AEkWM40yTr7yrXJOcDW'
attributes = 'ghi,dhi,dni,wind_speed,air_temperature'
leap_year = 'false'
interval = '30'
utc = 'false'
your_name = 'Frank+Waggoner'
reason_for_use = 'School+Project'
your_affiliation = 'CU+Boulder'
your_email = 'frwa2461@colorado.edu'
mailing_list = 'false'

# Get template inputs from the JSON file. File exported from SAM PV Model Template.sam
with open(json_file_name, 'r') as f:
        pv_inputs = json.load( f )
for k, v in pv_inputs.items():
    if k != 'number_inputs':
        pv_model.value(k, v)        

# Import generator data
PV_input_data = pd.read_csv(generator_inputs_location, index_col=None)
PV_capacities = PV_input_data.iloc[:,1].tolist()
PV_lats = PV_input_data.iloc[:,2].tolist()
PV_lons = PV_input_data.iloc[:,3].tolist()
PV_number_gens = len(PV_input_data.iloc[:,1])
years = [2012]

# Create output data structure
PV_total_output = []
header_list = ['Year','Month','Day','Hour','Minute','Total PV Output (kW)']

for i in range(PV_number_gens):
    # Change the value of iterative weather file variables
    lat = PV_lats[i]
    lon = PV_lons[i]
    year = years[0]
    # Declare url string for weather file.
    url = 'https://developer.nrel.gov/api/nsrdb/v2/solar/psm3-download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api_key, attr=attributes)
    df = pd.read_csv(url)
    # Create weather csv file
    df.to_csv(PV_weather_file_location)
    # Change the value of iterative PV system variables
    pv_model.value('solar_resource_file', PV_weather_file_location)
    pv_model.value('system_capacity', PV_capacities[i])
    # Run the module
    pv_model.execute()
    # Store power output data in a cumulative list
    for j in range(len(pv_model.Outputs.gen)):
        # Build empty total output list to store total output time series data
        # Add time series data to empty list
        if i == 0:
            PV_total_output.append([0,0,0,0,0,0])
            PV_total_output[j][0] = df.iat[j+2,0]
            PV_total_output[j][1] = df.iat[j+2,1]
            PV_total_output[j][2] = df.iat[j+2,2]
            PV_total_output[j][3] = df.iat[j+2,3]
            PV_total_output[j][4] = df.iat[j+2,4]
        # Fill in cumulative gen output time series data
        PV_total_output[j][5] = PV_total_output[j][5] + pv_model.Outputs.gen[j]
    # Print('Annual AC output for {capacity:,.2f} kW system is {output:,.0f} kWh.'.format(capacity = pv_model.value('system_capacity'), output = pv_model.Outputs.ac_annual) )
outputDF = pd.DataFrame(PV_total_output)
outputDF.to_csv(output_data_location, index=False, header=header_list)
print('Done')

32.7904
-117.0955


C:\Users\fwagg\AppData\Local\Temp\ipykernel_28464\1834736460.py:56: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,25,26,27,28,29,30,31,32,33,34,35,36,37,39,40,41,42,43,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


1100
32.878
-117.2362


C:\Users\fwagg\AppData\Local\Temp\ipykernel_28464\1834736460.py:56: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,25,26,27,28,29,30,31,32,33,34,35,36,37,39,40,41,42,43,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


1200
33.2495
-117.0216


...

1400
33.1391
-117.2314


C:\Users\fwagg\AppData\Local\Temp\ipykernel_28464\1834736460.py:56: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,25,26,27,28,29,30,31,32,33,34,35,36,37,39,40,41,42,43,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


1800
32.8337
-117.1288


C:\Users\fwagg\AppData\Local\Temp\ipykernel_28464\1834736460.py:56: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,25,26,27,28,29,30,31,32,33,34,35,36,37,39,40,41,42,43,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


2100
Done
